# 一、导入数据及数据预处理

In [2]:
import pandas as pd
import numpy as np
from pyecharts.charts import Pie,Bar,Line,Map,Map3D,Funnel
from pyecharts import options as opts
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType, ChartType
import textwrap
# 中文设置
plt.rcParams['font.sans-serif']=['Microsoft YaHei']
plt.rcParams['axes.unicode_minus']=False
plt.rc('font',family = 'Microsoft YaHei',size = '15')
warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
df = pd.read_csv('tmall_order.csv')
df.head()

,订单编号,总金额,买家实际支付金额,收货地址,订单创建时间,订单付款时间,退款金额
0,1,178.8,0.0,上海,2020-02-21 00:00:00,NaN,0.0
1,2,21.0,21.0,内蒙古自治区,2020-02-20 23:59:54,2020-02-21 00:00:02,0.0
2,3,37.0,0.0,安徽省,2020-02-20 23:59:35,NaN,0.0
3,4,157.0,157.0,湖南省,2020-02-20 23:58:34,2020-02-20 23:58:44,0.0
4,5,64.8,0.0,江苏省,2020-02-20 23:57:04,2020-02-20 23:57:11,64.8


In [5]:
# 去除字段名中的空格
new_columns = [col.strip() for col in df.columns]
df.columns = new_columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28010 entries, 0 to 28009
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   订单编号      28010 non-null  int64  
 1   总金额       28010 non-null  float64
 2   买家实际支付金额  28010 non-null  float64
 3   收货地址      28010 non-null  object 
 4   订单创建时间    28010 non-null  object 
 5   订单付款时间    24087 non-null  object 
 6   退款金额      28010 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.5+ MB


In [6]:
# 数据基本描述
print('数据的时间区间为',df['订单创建时间'].min(),'到',df['订单创建时间'].max())
print('收货地址总计有：',df['收货地址'].nunique(),'个')
df.describe()

数据的时间区间为 2020-02-01 00:14:15 到 2020-02-29 23:59:18
收货地址总计有： 31 个


,订单编号,总金额,买家实际支付金额,退款金额
count,28010.000000,28010.000000,28010.000000,28010.000000
mean,14005.500000,106.953253,67.921712,20.433271
std,8085.934856,1136.587094,151.493434,71.501963
min,1.000000,1.000000,0.000000,0.000000
25%,7003.250000,38.000000,0.000000,0.000000
50%,14005.500000,75.000000,45.000000,0.000000
75%,21007.750000,119.000000,101.000000,0.000000
max,28010.000000,188320.000000,16065.000000,3800.000000


In [7]:
# 提取日期中的时间为后续分析做准备
df['订单创建时间'] = df['订单创建时间'].astype('datetime64')
df['订单付款时间'] = df['订单付款时间'].astype('datetime64')
df['月'] = df['订单付款时间'].dt.month
df['日'] = df['订单付款时间'].dt.day
df2 = df[~df['订单付款时间'].isnull()]
df2['月'] = df2['月'].apply(lambda x:int(x)).astype('str')
df2['日'] = df2['日'].apply(lambda x:int(x)).astype('str')
df2['日期'] = df2['月'] + '月' + df2['日'] + '日'
df2['周'] = df2['订单付款时间'].dt.weekday + 1
df2['周'] = '星期' + df2['周'].astype('str')
df2['月'] = df2['月'].astype('int')
df2['日'] = df2['日'].astype('int')
df2 = df2.sort_values(by = '订单付款时间')
df2['小时'] = df2['订单付款时间'].dt.hour
df2.head()

ValueError: Passing in 'datetime64' dtype with no precision is not allowed. Please pass in 'datetime64[ns]' instead.

In [ ]:
# 查看收货地址信息
df2.收货地址.unique()

收货地址字段中，数据都为“四川省”、“西藏自治区”的形式，且四个直辖市是以“北京”、“上海”的数据形式，因为Pyecharts在绘图时候，数据中不能包含“省”、“自治区”这样的字眼，因此这里将收货地址数据做处理。

In [ ]:
df2['收货地址'] = df2.收货地址.apply(lambda x:x.strip('省|自治区'))
df2['收货地址'] = df2.收货地址.replace(['新疆维吾尔','广西壮族','宁夏回族'],['新疆','广西','宁夏'])
df2.head()
df2.收货地址.unique()

In [ ]:
# 查看缺失数据
df[df['订单付款时间'].isnull()].head()

In [ ]:
# 查看是否有重复值
df[df['退款金额'] > df['总金额']]
print('重复值数量为：',df.duplicated().sum())

In [ ]:
def kde_plot_array(df):
    """
    绘制概率密度图矩阵函数
    df:要绘制图像的dataframe
    绘制各个字段的概率密度分布，最终返回图像的show()
    """
    plt.figure(figsize = (24,16))  # 图一figure
    # plt.figure(figsize = (24,12))    # 图二figure
    # subplots_adjust(left = 0,bottom = 0,top = 1.4,right = 1)
    for num,columns in zip(range(len(df.columns)),df.columns):
        plt.subplot(round(len(df.columns)/2,0),2,num+1)
        # sns.set(font = 'SimHei',font_scale = 1.6)
        index = columns
        sns.kdeplot(df[columns],
                   shade = True,label = index,alpha = 0.7)
        plt.legend()
        plt.title('{}分布图'.format(index))
    return plt.show()

kde_plot_array(df[['总金额','买家实际支付金额','退款金额']])

通过绘制【总金额】、【买家实际支付金额】、【退款金额】的分布图，可以发现，这几个字段均有金额特别高的数据。有特别高的异常值，会影响我们对大部分订单金额分布的判断。  
因此接下来我们截取总金额小于500，退款金额小于400的数据，绘制概率密度图，查看大部分数据都分布在哪个数值附近。

In [ ]:
df.describe()
df[df.总金额 > 5000]
plot_df = df[(df.总金额 < 500)&(df.退款金额 < 400)][['总金额','买家实际支付金额','退款金额']]
kde_plot_array(plot_df)

绘制截取后的金额分布后我们可以发现如下信息：  
1. 总金额分布在50元和100元附近。单笔订单总金额为0-100元的居多。  
2. 买家实际支付金额和总金额分布基本一致，因为有退款行为的发生，会导致买家实际支付金额要比总金额分布更靠左一点。  
3. 退款金额分布在25-125元附近，因为单笔订单的购买金额通常并不会很高，因此退款金额的分布也不会很靠右。  
接下来我们看一下总金额特别高的几笔订单。

In [ ]:
df[df.总金额 > 3000]

查看总金额很大的几笔订单可以发现，最大的一笔订单为订单编号19258的订单，总金额为188320元，但是这笔订单没有完成交易，实际支付金额为0，因此判断可能是误操作导致。另外还有一万块以上的订单，也并没有发生未支付取消订单的情况。因此说明也会有这种订单金额很高的订单成交。因此对于这样的数据，并不进行异常值的处理，在后续分析中，将使用买家实际支付金额作为主要分析目标。

1. 缺失值  
在进行数据预处理时，发现数据仅【订单付款时间】字段有3923的缺失值，经过查看，发现订单付款时间缺失的数据中，【买家实际支付金额】都为0，且【退款金额】都是0，说明这样的订单都是没有付款的，取消交易的订单类型。这样的订单也属于正常现象，因此并不做处理。  
2. 重复值  
查看发现数据并无重复值。  
3. 异常值  
数据中只有一条总金额很大的异常数据，怀疑是误操作导致。但是后续分析也不会用到总金额，因此这样的异常数据同样也是不做处理。

# 二、数据分析及可视化

在处理完数据后，接下来要确定分析目标。因为常有退款现象发生、以及交易未完成等现象，因此我们只用实际支付金额作为分析目标，截取没有实际支付金额为0的数据。而分析这份数据，我决定从成交金额、成交次数、转换率这几部分作为分析目标来分析，并且从时间维度和空间维度作为分析维度，分别来看各指标，看一看能否发现一些有价值的信息。  
1. 成交金额  
	a. 成交金额在时间维度上的变化  
	b. 成交金额在地区维度上的分布  
2. 退款金额  
	a. 退款金额在时间维度上的变化  
	b. 退款金额在地区维度上的分布  
3. 转换率  
 a. 整体转换率  
 b. 全国各省市创建-支付转换率  
 c. 全国各省市支付-成交转换率

## 1. 成交金额

### a. 成交金额在时间维度上的变化

In [ ]:
change = df2[['买家实际支付金额','日']].groupby('日').sum().round(2).reset_index().sort_values(by = '日')

把要绘制的折线图调整好样式后，封装成这样的函数，然后后边在绘制类似的折线图的时候，就可以直接调用这个函数了，方便的很。由于绘制折线图部分的代码属实太长了，为不影响对整体报告的阅读，只好隐藏掉了。需要代码的，大家直接fork我的项目就可以看到啦~

In [ ]:

def echarts_line(x,y,title = '主标题',subtitle = '副标题',label = '图例'):
    """
    x: 函数传入x轴标签数据
    y：函数传入y轴数据
    title：主标题
    subtitle：副标题
    label：图例
    """
    line = Line(
        init_opts=opts.InitOpts(
            bg_color='#FFFFFF',  # 设置背景颜色
            theme='white'         # 设置主题
            # width='980px',     # 设置图的宽度
            # height='800px'     # 设置图的高度
        )
    )
    line.add_xaxis(x)
    line.add_yaxis(
        label,
        y,
        is_symbol_show=False,  # 是否显示数据标签点
        is_smooth=True,        # 设置曲线平滑
        label_opts=opts.LabelOpts(
            is_show=False,     # 是否显示数据
        ),
        itemstyle_opts=opts.ItemStyleOpts(color='#00ca95'),  # 设置系列颜色
        # 线条粗细阴影设置
        linestyle_opts={
            "normal": {
                "color": "#4ADEDE",  #线条颜色
                "shadowColor": 'rgba(0, 0, 0, .3)', #阴影颜色和不透明度
                "shadowBlur": 2,     #阴影虚化大小
                "shadowOffsetY": 5,  #阴影y偏移量
                "shadowOffsetX": 5,  #阴影x偏移量
                "width": 6   # 线条粗细
            },
        },
        # 阴影设置
        areastyle_opts={
            "normal": {
                "color": JsCode("""new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                                offset: 0,
                                color: '#7BD5F5'
                            },
                            {
                                offset: 1,
                                color: 'rgba(0,202,149, 0)'
                            }
                        ], false)"""),  #设置底色色块渐变
                "shadowColor": 'rgba(0,202,149, 0.9)',  #设置底色阴影
                "shadowBlur": 20  #设置底色阴影大小
            }
        },
    )
    line.set_global_opts(
        # 标题设置
        title_opts=opts.TitleOpts(
            title=title, # 主标题
            subtitle=subtitle, # 副标题
            pos_left='center',  # 标题展示位置
            title_textstyle_opts=dict(color='#fff') # 设置标题字体颜色
        ),
        # 图例设置
        legend_opts=opts.LegendOpts(
            is_show=True, # 是否显示图例
            pos_left='right', # 图例显示位置
            pos_top='3%',  #图例距离顶部的距离
            orient='horizontal'  # 图例水平布局
        ),
    )
    return line.render_notebook()


In [ ]:
echarts_line(change['日'],change['买家实际支付金额'],title = '成交金额变化图',subtitle = "成交金额在时间维度上的变化",
            label = '成交金额')

绘制成交金额在时间维度上的变化图，可以看到在2020年2月份的天猫成交金额在15日以前都是呈现一个较低的状态，而从15日开始，出现一个稳步上升的趋势，并且在22日和25日的时候分别达到了高峰。2022年的1月25日是春节，因此怀疑导致成交金额如此低下的原因，可能是受到春节影响，大部分消费者和商家都在停业过春节，而随着春节的结束，大部分人都开始逐渐进行了复工，因此从15日开始，成交金额开始逐步恢复正常，并且稳步提升。

In [ ]:
week_change = df2[['周','买家实际支付金额']].groupby('周').sum().round(2).reset_index()

In [ ]:

def echarts_bar(x,y,title = '主标题',subtitle = '副标题',label = '图例'):
    """
    x: 函数传入x轴标签数据
    y：函数传入y轴数据
    title：主标题
    subtitle：副标题
    label：图例
    """
    bar = Bar(
            init_opts=opts.InitOpts(
            bg_color='#FFFFFF',  # 设置背景颜色
            theme='white'         # 设置主题
            # width='980px',     # 设置图的宽度
            # height='800px'     # 设置图的高度
        )
    )
    bar.add_xaxis(x)
    bar.add_yaxis(label,y,
        label_opts=opts.LabelOpts(is_show=True) # 是否显示数据
        ,category_gap="50%" # 柱子宽度设置
        ) 
    bar.set_series_opts( # 自定义图表样式
        label_opts=opts.LabelOpts(is_show=False), # 是否显示数据标签
        markpoint_opts=opts.MarkPointOpts(
            data=[
            opts.MarkPointItem(type_="min", name="最小值"), # 显示最小值标签
            opts.MarkPointItem(type_="max", name="最大值"), # 显示最大值标签
            opts.MarkPointItem(type_="average", name="平均值") # 显示均值标签
            ]
        ),
        itemstyle_opts={  
            "normal": {
                "color": JsCode(
                    """new echarts.graphic.LinearGradient(0, 0, 0, 1, [{
                        offset: 0,color: 'rgba(0, 244, 255, 1)'}
                        ,{offset: 1,color: 'rgba(0, 77, 167, 1)'}], false)
                    """
                ),       # 调整柱子颜色渐变
                "barBorderRadius": [100, 100, 100, 100],  # 调整柱子圆角弧度
                "shadowColor": "rgb(0, 160, 221)", # 调整阴影颜色
            }
        }
    )
    bar.set_global_opts(
        # 标题设置
        title_opts=opts.TitleOpts(
            title=title, # 主标题
            subtitle=subtitle, # 副标题
            pos_left='center',  # 标题展示位置
            title_textstyle_opts=dict(color='#080b30') # 设置标题字体颜色
        ),
        # 图例设置
        legend_opts=opts.LegendOpts(
            is_show=True, # 是否显示图例
            pos_left='right', # 图例显示位置
            pos_top='3%',  #图例距离顶部的距离
            orient='horizontal'  # 图例水平布局
        ),
    )
    return bar.render_notebook()

In [ ]:
echarts_bar(week_change['周'].tolist(),week_change['买家实际支付金额'].tolist(),title = '订单成交金额平均每周对比',
            subtitle = '每周对比图',label = '成交金额')

绘制成交金额每周的对比图可以发现，周六日和周一的成交金额通常来说是较少的，每周的周二和周五的成交金额反而是较多的。猜测是因为周末大家通常来说都比较想好好安排一下自己的周末的生活，出去玩或者直接去实体店进行逛街购物，而平时工作日的时候，由于要忙工作的原因，因此只能进行线上网购，因此工作日期间网购的成交金额会相对来说较高一点。

In [ ]:
hour_change = df2[['小时','买家实际支付金额']].groupby('小时').sum().round(2).reset_index()

因为前边已经封装好了绘制折线图的函数，接下来就只要吧数据传进函数，一行代码直接绘图，肥肠好用~

In [ ]:
echarts_line(hour_change['小时'],hour_change['买家实际支付金额'],title = '每天各时段成交金额变化图'
            ,subtitle = '一天24小时哪个时间段成交金额多',label = '成交金额')

绘制每天各时段的成交金额变化图，可以发现，每天的早上10点，下午的15点，晚上的21-23点的时间段，是成交金额的高峰期。尤其是晚上这个时间段，成交金额相对来说是最高的。这可能因为人在晚上是比较容易冲动消费的，也可能因为大家再进行了一整天的工作后，只有在晚上吃完饭，忙完所有工作以后，才有时间来天猫逛一逛，进行一些网购等。而10点和15点通常也都是大家吃完早餐和午餐以后，进行了一段时间的工作以后，稍微休息一下，进行一些网购行为（猜的奥~）。

### b. 成交金额在地区维度上的分布

In [ ]:
change_map = df2[['收货地址','买家实际支付金额']].groupby('收货地址').sum().round(2).reset_index().sort_values(by = '买家实际支付金额',
                ascending = False)

In [ ]:

def echarts_map(province,data,title = '主标题',subtitle = '副标题',label = '图例'):
    """
    province：传入省份List
    data：传入各省对应的数据List
    title：主标题
    subtitle：副标题
    label：图例
    """
    map_ = Map(
            init_opts=opts.InitOpts(
            bg_color='#FFF',  # 设置背景颜色
            theme='white'         # 设置主题
            # width='980px',     # 设置图的宽度
            # height='800px'     # 设置图的高度
            )
    )
    map_.add(label,[list(i) for i in zip(province,data)])
    map_.set_global_opts(
        # 标题设置
        title_opts=opts.TitleOpts(
            title=title, # 主标题
            subtitle=subtitle, # 副标题
            pos_left='center',  # 标题展示位置
            title_textstyle_opts=dict(color='#fff') # 设置标题字体颜色
        ),
        # 图例设置
        legend_opts=opts.LegendOpts(
            is_show=True, # 是否显示图例
            pos_left='right', # 图例显示位置
            pos_top='3%',  #图例距离顶部的距离
            orient='horizontal'  # 图例水平布局
        ),
        visualmap_opts = opts.VisualMapOpts(max_ = int(data.max()),is_piecewise = False)
    )
    return map_.render_notebook()


In [ ]:
echarts_map(change_map['收货地址'],change_map['买家实际支付金额'],title = '成交金额分布图'
            ,subtitle = '成交金额在全国各地分布情况',label = '成交金额')

绘制成交金额在全国各地的分布，我们可以发现江浙沪一带、以及北京、广东、四川等地区的成交金额是比较高的，而新疆西藏内蒙古等其他地区的成交金额普遍较低。北上广深等一线城市本身就是人口较多、经济较发达的城市，所以成交金额自然也会较高。

In [ ]:
change_map = df2[['收货地址','买家实际支付金额']].groupby('收货地址').sum().round(2).reset_index().sort_values(by = '买家实际支付金额',
                ascending = False)

In [ ]:

def map3d_with_bar3d(province,data_lis,title = '主标题',subtitle = '副标题',label = '图例'):
    """
    province：传入省份的List
    data_lis：传入省份对应的数据的List
    title：主标题
    subtitle：副标题
    label：图例
    """
    data_dict = {'黑龙江': [127.9688, 45.368], '上海': [121.4648, 31.2891],
            '内蒙古': [110.3467, 41.4899], '吉林': [125.8154, 44.2584],
            '辽宁': [123.1238, 42.1216], '河北': [114.4995, 38.1006],
            '天津': [117.4219, 39.4189], '山西': [112.3352, 37.9413],
            '陕西': [109.1162, 34.2004], '甘肃': [103.5901, 36.3043],
            '宁夏': [106.3586, 38.1775], '青海': [101.4038, 36.8207],
            '新疆': [87.9236, 43.5883], '西藏': [91.11, 29.97],
            '四川': [103.9526, 30.7617], '重庆': [108.384366, 30.439702],
            '山东': [117.1582, 36.8701], '河南': [113.4668, 34.6234],
            '江苏': [118.8062, 31.9208], '安徽': [117.29, 32.0581],
            '湖北': [114.3896, 30.6628], '浙江': [119.5313, 29.8773],
            '福建': [119.4543, 25.9222], '江西': [116.0046, 28.6633],
            '湖南': [113.0823, 28.2568], '贵州': [106.6992, 26.7682],
            '广西': [108.479, 23.1152], '海南': [110.3893, 19.8516],
            '广东': [113.28064, 23.125177], '北京': [116.405289, 39.904987],
            '云南': [102.71225, 25.040609], '香港': [114.165460, 22.275340],
            '澳门': [113.549130, 22.198750], '台湾': [121.5200760, 25.0307240]}
    for item in [list(z) for z in zip(province, data_lis)]:
        data_dict[item[0]].append(item[1])
    example_data = list(zip(data_dict.keys(),data_dict.values()))

    map_3d = Map3D(
        init_opts=opts.InitOpts(
        bg_color='#080b30',  # 设置背景颜色
        theme='dark',         # 设置主题
        # width='1200px',     # 设置图的宽度
        # height='1000px'     # 设置图的高度
        )
    )
    map_3d.add_schema(
        itemstyle_opts=opts.ItemStyleOpts(
            color="rgb(5,101,123)",
            opacity=1,
            border_width=0.8,
            border_color="rgb(62,215,213)",
        ),
        map3d_label=opts.Map3DLabelOpts(
            is_show=False,
            formatter=JsCode(
                "function(data){return data.name + " " + data.value[2];}"
            ),
        ),
        emphasis_label_opts=opts.LabelOpts(
            is_show=False,
            color="#fff",
            font_size=10,
            background_color="rgba(0,23,11,0)",
        ),
        light_opts=opts.Map3DLightOpts(
            main_color="#fff",
            main_intensity=1.2,
            main_shadow_quality="high",
            is_main_shadow=False,
            main_beta=10,
            ambient_intensity=0.3,
        )
    )
    map_3d.add(
        series_name=label,
        data_pair=example_data,
        type_=ChartType.BAR3D,
        bar_size=1,
        shading="lambert",
        label_opts=opts.LabelOpts(
            is_show=True,
            formatter=JsCode(
                "function(data){return data.name + ' ' + data.value[2];}"
            ),
        ),
    )
    map_3d.set_global_opts(
    # 标题设置
        title_opts=opts.TitleOpts(
            title= title, # 主标题
            subtitle= subtitle, # 副标题
            pos_left='center',  # 标题展示位置
            title_textstyle_opts=dict(color='#fff') # 设置标题字体颜色
        ),
    # 图例设置
        legend_opts=opts.LegendOpts(
            is_show=True, # 是否显示图例
            pos_left='right', # 图例显示位置
            pos_top='3%',  #图例距离顶部的距离
            orient='horizontal'  # 图例水平布局
        ),
    )
    return map_3d.render_notebook()

In [ ]:
map3d_with_bar3d(change_map['收货地址'],change_map['买家实际支付金额'],title = '成交金额分布图'
            ,subtitle = '成交金额在全国各地分布情况',label = '成交金额')

因为pyecharts在新版本中更新了Map3D的3D地图，个人比较喜欢这个效果，所以这里也是来尝试一下。其实这个图还是蛮好绘制的，并且结合柱状图，可以很直观的看到哪里的销售额较高，通常来说做一些数字化看板等会用的比较多，因为结果较为炫酷嘛。

## 2. 退款金额

### a. 退款金额在时间维度上的变化

In [ ]:
back_money = df2[['日','退款金额']].groupby('日').sum().round(2).reset_index()

In [ ]:
echarts_bar(back_money['日'].tolist(),back_money['退款金额'].tolist(),title = '退款金额变化图'
            ,subtitle = '退款金额日变化图',label = '退款金额')

绘制退款金额在时间维度上的变化图可以发现，和成交金额几乎一致，都是在25号达到了一个高峰的状态，成交的金额多的时候，退款的金额也相应的多。

In [ ]:
hour_back_money = df2[['小时','退款金额']].groupby('小时').sum().round(2).reset_index()

In [ ]:
echarts_line(hour_back_money['小时'].tolist(),hour_back_money['退款金额'].tolist(),title = '退款金额变化图'
                ,subtitle = '退款金额每日小时变化图',label = '退款金额')

绘制退款金额在每天各小时的变化图，发现和成交金额的变化也差不多，都是在10点、15点、22点的时间段会比较多。

### b. 退款金额在地区维度上的分布

In [ ]:
local_back_money = df2[['收货地址','退款金额']].groupby('收货地址').sum().round(2).reset_index()

In [ ]:
echarts_map(local_back_money['收货地址'],local_back_money['退款金额'],title = '退款金额分布图'
            ,subtitle = '退款金额在全国各地分布情况',label = '退款金额')

In [ ]:
map3d_with_bar3d(local_back_money['收货地址'],local_back_money['退款金额'],title = '退款金额分布图'
            ,subtitle = '退款金额在全国各地分布情况',label = '退款金额')

## 3. 转换率

### a. 整体转换率

In [ ]:
# 计算各阶段订单量
create_order = df.订单编号.count()
print('下单数量为：',create_order)
buy_order = df[~df.订单付款时间.isnull()].订单编号.count()
print('支付订单数量为：',buy_order)
finish_order = df[df.买家实际支付金额 > 0].订单编号.count()
print('成交订单数量为：',finish_order)
whole_finish_order = df[df.总金额 == df.买家实际支付金额].订单编号.count()
print('全额成交订单量为：',whole_finish_order)

In [ ]:
# 计算各阶段转化率
transform = [['创建订单',round(create_order / df.shape[0],2)],
             ['支付订单',round(buy_order / df.shape[0],2)],
             ['成交订单',round(finish_order / df.shape[0],2)],
             ['全额成交订单',round(whole_finish_order / df.shape[0],2)]]
transform

In [ ]:
funnel=Funnel(
        init_opts=opts.InitOpts(
        bg_color='#FFFFFF',  # 设置背景颜色
        theme='wgite',         # 设置主题
        # width='980px',     # 设置图的宽度
        # height='800px'     # 设置图的高度
        )
)
funnel.add("销售图",transform
        ,is_selected=True
        ,label_opts=opts.LabelOpts(position='inside', formatter='{b}：{c}'), # 设置文字显示在漏斗图内
    )
funnel.set_global_opts(
        # 标题设置
        title_opts=opts.TitleOpts(
            title='天猫2月销售订单转化率漏斗图', # 主标题
            subtitle=' ', # 副标题
            pos_left='center',  # 标题展示位置
            title_textstyle_opts=dict(color='#fff') # 设置标题字体颜色
        ),
        # 图例设置
        legend_opts=opts.LegendOpts(
            is_show=True, # 是否显示图例
            pos_left='right', # 图例显示位置
            pos_top='3%',  #图例距离顶部的距离
            orient='horizontal'  # 图例水平布局
        )
    )
funnel.render_notebook()

1. 通过销售额漏斗图可以发现，成交订单到全额成交订单的转化率还是蛮高的，只下降了2个百分点，这说明通常来说只要订单成交了，那么基本上就是进行的全额成交，只会有小部分消费者有部分退款行为。  
2. 创建订单到支付订单的转化率有86%，这说明有14%的用户在创建了订单以后，没有选择支付，而是取消了该笔订单的支付。应当重点追溯一下导致这个情况的原因是什么。可以查询创建订单以后的用户行为都有哪些，创建了订单以后，用户是立即取消了订单支付？还是用户离开了支付界面进行了其他行为？用户离开支付界面，都进行了哪些行为，是继续浏览同类商品？还是退出app忘记支付？是因为看到了其他同类更好的商品决定不购买该商品了？还是只是点错了，不小心进到了支付界面？还是想要购买的商品规格不符合自己的预期？提升创建订单到支付订单的转化率是较为关键的。  
3. 支付订单到成交订单的转化率同样比较低，转换率下降了18%。这就说明，有18% 的用户在支付了订单金额以后，选择了全额退款。这个问题也不容小觑。应当查明导致该问题的原因。是购买的产品规格不符合预期需要重新下单？还是收到了商品以后发现有质量问题，需要全额退款？还是因为物流速度太慢，导致的全额退款行为？在整个销售漏斗中，这部分的转化率是最低的，因此应当结合其他数据，着重查找该环节的原因，从而提升销售额。

### b. 全国各省市创建-支付转换率

通过前边一部分的分析，我们知道，在创建 -> 支付和支付 -> 成交的转换率是较低的。因此我们猜测，是否受到地区影响，导致物流等问题，发生了取消支付或者是退货等情况。因此接下来我们分别查看全国各省市创建-支付转换率和全国各省市支付-成交转换率，看看到底是哪个省市的转换率较低。

In [ ]:
# 全国各省市创建-支付转换率计算
df['辅助计数'] = 1
pay_order = df[~df.订单付款时间.isnull()]
province_pay_order = pay_order[['收货地址','辅助计数']].groupby('收货地址').sum().reset_index()
create_order = df[['收货地址','辅助计数']].groupby('收货地址').sum().reset_index()
province_create_pay_transform = pd.merge(create_order,province_pay_order,how = 'left',on = '收货地址')
province_create_pay_transform.rename(columns = {'辅助计数_x':'创建订单数','辅助计数_y':'支付订单数'},inplace = True)
province_create_pay_transform['创建-支付转换率'] = province_create_pay_transform['支付订单数'] / province_create_pay_transform['创建订单数']
province_create_pay_transform['创建-支付转换率'] = province_create_pay_transform['创建-支付转换率'].round(2)
province_create_pay_transform.head()

In [ ]:
echarts_bar(province_create_pay_transform['收货地址'].tolist(),province_create_pay_transform['创建-支付转换率'].tolist(),
            title = '创建-支付转换率对比图'
            ,subtitle = '全国各省创建-支付转换率对比图',label = '创建-支付转换率')

1. 通过绘制全国各省市创建-支付转换率的对比图我们可以发现，大部分省份的转换率都是处于一个相对稳定的水平，都处在均值0.84附近，而宁夏、上海、青海、北京等地区的转换率更是高达90%以上的，说明大部分地区的用户在支付界面取消订单的现象并不严重。  
2. 有三个特别的地区，创建-支付订单的转化率较低，分别是西藏、新疆和湖北。西藏和新疆因为处于偏远地区，交通并不是很便利，因此物流是比较慢的，很多商家都是不往这两个地区发货的，因此可能受到物流影响，转换率会较低。而湖北地区的转化率较低，很可能是因为受到疫情影响。2020年的2月份正是湖北疫情严重的时期，因此很多消费者会选择取消订单，这也可能是转换率较低的原因。

### c. 全国各省市支付-成交转换率

In [ ]:
# 全国各省市支付-成交转换率计算
df2['辅助计数'] = 1
finish_df = df2[df2.买家实际支付金额 > 0]
province_finish_df = finish_df[['收货地址','辅助计数']].groupby('收货地址').sum().reset_index()
province_buy_df = df2[['收货地址','辅助计数']].groupby('收货地址').sum().reset_index()
province_transform = pd.merge(province_buy_df,province_finish_df,how = 'outer',on = '收货地址')
province_transform.rename(columns = {'辅助计数_x':'支付订单数','辅助计数_y':'成交订单数'},inplace = True)
province_transform['支付-成交转换率'] = province_transform['成交订单数'] / province_transform['支付订单数']
province_transform['支付-成交转换率'] = province_transform['支付-成交转换率'].round(2)
province_transform.head()

In [ ]:
echarts_bar(province_transform['收货地址'].tolist(),province_transform['支付-成交转换率'].tolist(),
            title = '支付-成交转换率对比图'
            ,subtitle = '全国各省支付-成交转换率对比图',label = '支付-成交转换率')

In [ ]:
province_transform[province_transform.收货地址 == '西藏']

In [ ]:
province_create_pay_transform[province_create_pay_transform.收货地址 == '湖北省']

In [ ]:
province_transform[province_transform.收货地址 == '湖北']

1. 绘制全国各省市支付-成交转换率对比图可以发现，该部分转换率较低的几个省份依旧是新疆、西藏和湖北。西藏的转换率为0，查询原因发现，西藏在2月份有过支付记录的订单总共就2个，因此西藏的转化率达到了0也不算异常。  
2. 而湖北的转换率达到了0.32，查询2月份湖北的订单创建数和支付数，发现湖北2月份创建的订单仅有75个，支付的57个，而真正成交的只有18个订单，从支付到成交的转换率仅有32%，说明有68%的用户直接进行了全额退款。这可能是因为2020年2月份湖北疫情较为严重，湖北的物流基本上呈现中断的状态，大部分商品很可能不在发往湖北，因此出现了大量的全额退款等行为。

# 三、总结与建议

1. 成交金额方面，新年过后复工的一周左右，销售额会出现快速的增长，在月底前达到一个小高峰。因此可以在新年过后一周左右的时间，提前准备好库存，做好销售计划等。  
2. 每周的周二和周五是销售额较高的时间段。如果有促销计划什么的，可以重点在周二和周五进行。  
3. 每日的10点、15点、21-22点的时间段是销售额较高的时间段，如果有促销活动、优惠券发放、新品推荐等推送内容，可以选择在这些时间段进行发放，效果会更佳。  
4. 地区方面，江浙沪一带、以及北京、广东、四川等地区的成交金额是比较高的，而新疆西藏内蒙古等其他地区的成交金额普遍较低。可以根据以后的发展计划、策略、综合考虑物流、成本等因素进行产品销售的调整。  
5. 退款金额和销售金额的趋势基本吻合。销售额高相应的退款行为发生次数也多。  
6. 转换率方面，创建订单->支付订单->成交订单的转化率均较低，应当从各方面寻找原因，加强物流管理、提升产品质量、提高服务态度等等，从而提高各环节转换率，增加销量。  
7. 对于转换率较低的省份，因受到疫情影响，因此转换率较低，出现大量退款等行为，一方面要全力配合防疫工作，另一方面要在疫情到来之前，提前做好应对措施，及时调整营销策略等。